In [108]:
import numpy as np
import pandas as pd

In [111]:
# Run this code in the chrome console on a when2meet to generate a csv text of the when2meet data. 
# Save it into an csv file to import it here
'''
function getCSV() {
  result = "Time," + PeopleNames.join(",")+"\n"; 
  for(let i = 0; i < AvailableAtSlot.length; i++) {
      let slot = $x(`string(//div[@id="GroupTime${TimeOfSlot[i]}"]/@onmouseover)`);
      slot = slot.match(/.*"(.*)".*/)[1];
      result += slot + ",";
      result += PeopleIDs.map(id => AvailableAtSlot[i].includes(id) ? 1 : 0).join(",");
      result+= "\n";
  }
  console.log(result);
}
getCSV();
'''


'\nfunction getCSV() {\n  result = "Time," + PeopleNames.join(",")+"\n"; \n  for(let i = 0; i < AvailableAtSlot.length; i++) {\n      let slot = $x(`string(//div[@id="GroupTime${TimeOfSlot[i]}"]/@onmouseover)`);\n      slot = slot.match(/.*"(.*)".*/)[1];\n      result += slot + ",";\n      result += PeopleIDs.map(id => AvailableAtSlot[i].includes(id) ? 1 : 0).join(",");\n      result+= "\n";\n  }\n  console.log(result);\n}\ngetCSV();\n'

In [75]:
dfraw = pd.read_csv('Schedule.csv')
dfraw.head()

,Time,Addie Foote,Aidan Sices,Alex Dial,Ananya,Ananya Choudhary,Claire Tsui,Claudia,Cody Rushing,Daniel Kazour,...,Jamie Ordonez,Juliana,Katherine Dial,Michael Montez,Michael Zhao,Rishi,Rutvij,Sara Ahmmed,Sidh Suchdev,Trinity
0,Sunday 11:00:00 AM,1,1,1,0,1,1,1,0,1,...,0,0,1,1,0,0,0,0,1,1
1,Sunday 11:15:00 AM,1,1,1,0,1,1,1,0,1,...,0,0,1,1,0,0,0,0,1,1
2,Sunday 11:30:00 AM,1,1,1,0,1,1,1,0,1,...,0,0,1,1,0,0,0,0,1,1
3,Sunday 11:45:00 AM,1,1,1,0,1,1,1,0,1,...,0,0,1,1,0,0,0,0,1,1
4,Sunday 12:00:00 PM,1,1,1,0,1,1,1,0,1,...,0,0,1,1,0,0,0,0,1,1


# Approach

We want to pick 3 rows such that 
    a) Each row has at least x many 1s and 
    b) every col has at least one 1

We will clean our data in 2 ways:
    1) delete all columns where faciltators are busy and where fewer than x many participants are available
    2) Delete all the 15 minute options

In [76]:
# Define who the facilitators are
facil = ['Cody Rushing', 'Alex Dial', 'Trinity']

# What is the minimum group size
min_size = 5

In [77]:
# drop the rows that have ":15:" or ":45:" in them, and format our times better
df = dfraw.copy()
#df.drop(df.index[[("135" in df.loc[i, 'Time'] or "453" in df.loc[i, 'Time'] or "Sat" in df.loc[i, 'Time'] or "Fri" in df.loc[i, 'Time'] or "Thur" in df.loc[i, 'Time']) for i in range(len(df))]], inplace=True)
df.drop(df.index[[("15" in df.loc[i, 'Time'] or "45" in df.loc[i, 'Time']) for i in range(len(df))]], inplace=True)

#df['Time'] = df['Time'].apply(lambda x: x[:4] + x[16:21] + x[24:27])


# Find the names of the people with like no availability who we should disregard
bad = df.columns[df.sum() == 0]
print('We are not including these people because they have no availability:')
[print(i) for i in bad]
print('--')

# find the names of the nonfacilitators who aren't bad
partic = list(set(df.columns[1:]) - set(facil) - set(bad))

# Clean df to just times where at least min_size good participants and at least one facilitator is available
cdf = df.loc[(df[partic].sum(axis=1) >= min_size) & (df[facil].sum(axis=1) >= 1)].drop(bad, axis=1)
cdf.set_index("Time", inplace=True)

# Move facilitators to the front
for fac in facil:
    first_column = cdf.pop(fac)
    cdf.insert(0, fac, first_column)

# Preview our cleaned data
cdf.head()

We are not including these people because they have no availability:
--


,Trinity,Alex Dial,Cody Rushing,Addie Foote,Aidan Sices,Ananya,Ananya Choudhary,Claire Tsui,Claudia,Daniel Kazour,Garrett,Jamie Ordonez,Juliana,Katherine Dial,Michael Montez,Michael Zhao,Rishi,Rutvij,Sara Ahmmed,Sidh Suchdev
Time,,,,,,,,,,,,,,,,,,,,
Sunday 11:00:00 AM,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1
Sunday 11:30:00 AM,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1
Sunday 12:00:00 PM,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1
Sunday 12:30:00 PM,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1
Sunday 01:00:00 PM,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1


In [78]:
# See the times with the most availability
cdf.sum(axis=1).sort_values(ascending=False).head(10)

Time
Saturday 02:00:00 PM    16
Saturday 02:30:00 PM    16
Saturday 03:00:00 PM    15
Saturday 01:30:00 PM    14
Sunday 06:00:00 PM      14
Saturday 03:30:00 PM    14
Sunday 08:00:00 PM      13
Sunday 07:30:00 PM      13
Sunday 05:00:00 PM      13
Sunday 04:30:00 PM      13
dtype: int64

In [79]:
# Firstly, check if there's anyone who already has no availability left
# sum(cdf.sum() == 0)
noFits = cdf.columns[cdf.sum() == 0]
print('Given our constraints, there are no times left that work for:')
[print(i) for i in noFits]
print('--')

Given our constraints, there are no times left that work for:
--


In [80]:
# What people only have very little availability (less than 20 time slots)
# cdf.sum()
lowAvail = cdf.columns[cdf.sum() <= 10]
print('These people have the least availability:')
[print(i) for i in lowAvail]
print('--')
cdf[lowAvail]

These people have the least availability:
Juliana
Rutvij
Sara Ahmmed
--


,Juliana,Rutvij,Sara Ahmmed
Time,,,
Sunday 11:00:00 AM,0,0,0
Sunday 11:30:00 AM,0,0,0
Sunday 12:00:00 PM,0,0,0
Sunday 12:30:00 PM,0,0,0
Sunday 01:00:00 PM,0,0,0
...,...,...,...
Saturday 06:30:00 PM,0,0,0
Saturday 07:00:00 PM,1,0,0
Saturday 07:30:00 PM,1,0,0


In [81]:
# function for testing a selection. return how many don't get a time, and how many go for each timeslot
def is_good(df):
    ans = list(df.index)
    # ans.append(df.columns[df.sum() == 0]) <- This will pass on the names of those who cant attend
    ans.append(sum(df.sum() == 0))
    ans.append(min(df.iloc[:, len(facil):].sum(axis=1)))
    ans.append(df.columns[df.sum() == 0])
    return ans

In [82]:
# create empty results df
def get_results(df, cohorts):
    cols = ['Time' + str(i+1) for i in range(cohorts)]
    cols.append('Absent')
    cols.append('minAtt')
    cols.append('who')
    results = pd.DataFrame(columns = cols)

    p = [0 for i in range(cohorts)]
    pos = len(p) - 1
    while p[0] < len(df):

        # add to the last position.
        # Check if the current position is filled. If it is, set it to the leftward position
        # now move current position to the left one. 
        results.loc[len(results)] = is_good(df.loc[df.index[p]])

        pos = len(p) - 1
        p[pos] += 1
        while p[pos] >= len(df) and pos > -1:
            # move backward one position
            pos -= 1
            p[pos] += 1

        while pos < len(p) - 1:
            pos += 1
            p[pos] = p[pos-1]
            
    # when you have some ppl that can attend multiple, prefer them to go to the smallest group. indicate this by just
    # presenting the groups in ascending order

    results.sort_values(['Absent', 'minAtt'], ascending=[True, False], inplace=True)
    results.reset_index(inplace=True, drop=True)

    return results

In [106]:
# Examine the time choice of a particular index
def inspect(df, results, idx):
    sel = df.loc[results.loc[idx][:-3]]
    '''
    print('Schedule possible for idx=' + str(idx) + ":")
    print('----------------------')
    for ses in range(len(sel)):
        print('Session ' + str(ses) + ' @ ' + sel.iloc[ses].name)
        print()
        print('%Facilitator:')
        curFacil = sel.iloc[ses][facil]
        curFacil = curFacil[curFacil == 1].index
        [print(i) for i in curFacil]
        print()
        curFell = sel.iloc[ses][partic]
        curFell = curFell[curFell == 1].index
        print('%Fellows:')
        print(len(curFell))
        [print(i) for i in curFell]
    
        print('------')
    '''
    print('The following people cant make any these times:')
    [print(i) for i in sel.columns[sel.sum() == 0]]
    print('--')
    
    sel.insert(0, 'Totals', sel.iloc[:, len(facil):].sum(axis=1))
    
    return sel

In [84]:
# Get the results for our cleaned dataset and view the 5 best selections
results = get_results(cdf, 3)
results.to_excel("UpdatedAllTimes.xlsx") 

In [85]:
# read in results file if we've already made one
#results = pd.read_excel('UpdatedAllTimes.xlsx')
#results.drop('Unnamed: 0', axis=1, inplace=True)

In [92]:
# Option 6 Looks good!
results.head(25)

,Time1,Time2,Time3,Absent,minAtt,who
0,Sunday 06:00:00 PM,Monday 04:30:00 PM,Saturday 01:30:00 PM,0,10,"Index([], dtype='object')"
1,Sunday 06:00:00 PM,Monday 04:30:00 PM,Saturday 02:00:00 PM,0,10,"Index([], dtype='object')"
2,Sunday 06:00:00 PM,Monday 04:30:00 PM,Saturday 02:30:00 PM,0,10,"Index([], dtype='object')"
3,Sunday 06:00:00 PM,Monday 04:30:00 PM,Saturday 03:00:00 PM,0,10,"Index([], dtype='object')"
4,Sunday 06:00:00 PM,Monday 04:30:00 PM,Saturday 03:30:00 PM,0,10,"Index([], dtype='object')"
5,Sunday 06:00:00 PM,Wednesday 06:00:00 PM,Saturday 01:30:00 PM,0,10,"Index([], dtype='object')"
6,Sunday 06:00:00 PM,Wednesday 06:00:00 PM,Saturday 02:00:00 PM,0,10,"Index([], dtype='object')"
7,Sunday 06:00:00 PM,Wednesday 06:00:00 PM,Saturday 02:30:00 PM,0,10,"Index([], dtype='object')"
8,Sunday 06:00:00 PM,Wednesday 06:00:00 PM,Saturday 03:00:00 PM,0,10,"Index([], dtype='object')"
9,Sunday 06:00:00 PM,Wednesday 06:30:00 PM,Saturday 01:30:00 PM,0,10,"Index([], dtype='object')"


In [107]:
inspect(cdf, results, 6)

The following people cant make any these times:
--


,Totals,Trinity,Alex Dial,Cody Rushing,Addie Foote,Aidan Sices,Ananya,Ananya Choudhary,Claire Tsui,Claudia,...,Garrett,Jamie Ordonez,Juliana,Katherine Dial,Michael Montez,Michael Zhao,Rishi,Rutvij,Sara Ahmmed,Sidh Suchdev
Time,,,,,,,,,,,,,,,,,,,,,
Sunday 06:00:00 PM,11,1,1,1,0,1,0,1,1,1,...,1,1,1,1,0,0,1,1,0,0
Wednesday 06:00:00 PM,10,1,1,1,1,1,0,1,0,1,...,0,0,0,1,1,0,1,0,1,1
Saturday 02:00:00 PM,13,1,1,1,1,1,1,0,1,1,...,1,1,0,1,1,1,1,0,0,1


In [88]:
# Doing some filterings to look for specific days since this week 1 facilitators are out of town
r = results.copy()
# add a column of the 3 time strings
r['cat'] = [r.loc[i, 'Time1'] + r.loc[i, 'Time2'] + r.loc[i, 'Time3'] for i in range(len(r))]

In [102]:
# Facilitators gone Saturday and we'd prefer to keep the Wed option the same and have the makeup ASAP in the next week
# Delete all saturday options, require a monday option, and require a Wednesday 6pm option
cr = r.drop(r.index[['Sat' in r.loc[i, 'cat'] or 'Mon' not in r.loc[i, 'cat'] or 'Wednesday 06:00:00 PM' not in r.loc[i, 'cat']for i in range(len(r))]])

In [90]:
cr.head(5)

,Time1,Time2,Time3,Absent,minAtt,who,cat
152,Sunday 06:00:00 PM,Monday 04:30:00 PM,Wednesday 06:00:00 PM,1,10,"Index(['Michael Zhao'], dtype='object')",Sunday 06:00:00 PMMonday 04:30:00 PMWednesday ...
534,Sunday 06:00:00 PM,Monday 04:00:00 PM,Wednesday 06:00:00 PM,1,8,"Index(['Michael Zhao'], dtype='object')",Sunday 06:00:00 PMMonday 04:00:00 PMWednesday ...
2448,Sunday 06:00:00 PM,Monday 05:30:00 PM,Wednesday 06:00:00 PM,2,9,"Index(['Ananya ', 'Michael Zhao'], dtype='obje...",Sunday 06:00:00 PMMonday 05:30:00 PMWednesday ...
2454,Sunday 06:00:00 PM,Monday 06:00:00 PM,Wednesday 06:00:00 PM,2,9,"Index(['Ananya ', 'Michael Zhao'], dtype='obje...",Sunday 06:00:00 PMMonday 06:00:00 PMWednesday ...
2467,Sunday 06:00:00 PM,Monday 06:30:00 PM,Wednesday 06:00:00 PM,2,9,"Index(['Ananya ', 'Michael Zhao'], dtype='obje...",Sunday 06:00:00 PMMonday 06:30:00 PMWednesday ...


In [103]:
inspect(cdf, results, 152)

The following people cant make any these times:
Michael Zhao
--


,Totals,Trinity,Alex Dial,Cody Rushing,Addie Foote,Aidan Sices,Ananya,Ananya Choudhary,Claire Tsui,Claudia,...,Garrett,Jamie Ordonez,Juliana,Katherine Dial,Michael Montez,Michael Zhao,Rishi,Rutvij,Sara Ahmmed,Sidh Suchdev
Time,,,,,,,,,,,,,,,,,,,,,
Sunday 06:00:00 PM,11,1,1,1,0,1,0,1,1,1,...,1,1,1,1,0,0,1,1,0,0
Monday 04:30:00 PM,10,0,0,1,1,1,1,1,1,0,...,0,1,0,0,1,0,1,0,1,0
Wednesday 06:00:00 PM,10,1,1,1,1,1,0,1,0,1,...,0,0,0,1,1,0,1,0,1,1
